## Pascal VOC Dataset Tensorflow Object Detection API Project

## Choosing a Pretrained Model
The model that will be trained in this Colab notebook is `rfcn_resnet101`.
Link to [all pretrained models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md).

In [ ]:


# Models to be trained in this project
MODELS_CONFIG = {
    'faster_rcnn_resnet101': {
        'model_name': 'faster_rcnn_resnet101_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_resnet101_voc07.config',
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_coco.config',
    },
    'ssd_inception_v2': {
        'model_name': 'ssd_inception_v2_coco_2018_01_28',
        'pipeline_file': 'ssd_inception_v2_coco.config',
    }
}

# We will train R-FCN in this Colab notebook
selected_model = 'rfcn_resnet101'


## Installing Required Packages and Importing Required Modules

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!apt-get install -qq protobuf-compiler

In [ ]:
import os
import shutil

import urllib.request
import tarfile

from google.colab import files

## Downloading TensorFlow Model Garden and Preparing TensorFlow Object Detection API
1. Clones [TensorFlow Model Garden](https://github.com/tensorflow/models.git) from the offical git repo. The repo contains the object detection API we are interested in. 
2. Tests the model builder.

In [ ]:
# Downloads TensorFlow Model Garden
!git clone --q https://github.com/tensorflow/models.git

In [ ]:
# Installs the Object Detection API
%%bash
cd models/research
# Compiles protos.
protoc object_detection/protos/*.proto --python_out=.
# Installs TensorFlow Object Detection API.
cp object_detection/packages/tf1/setup.py .
python -m pip install .

In [ ]:
# Tests the model builder
!python /content/models/research/object_detection/builders/model_builder_tf1_test.py

## Downloading the Dataset and Generating TFRecords
1. Downloads the dataset from the [source](http://host.robots.ox.ac.uk/pascal/VOC/)  and extracts them.
2. Generates TFRecords for training and validation image files.

In [ ]:
# Downloads the dataset

# Source: http://host.robots.ox.ac.uk/pascal/VOC/
# Source2: https://data.deepai.org/PASCALVOC2007.zip

# Changes directory
os.chdir("/content")

# Downloads the dataset tar file
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar

# Extracts the dataset
tar = tarfile.open('VOCtrainval_06-Nov-2007.tar')
tar.extractall()
tar.close()

# Deletes unneeded tar file
!rm -rf VOCtrainval_06-Nov-2007.tar

In [ ]:
# Generates TFRecord for the training and validation image files

# Changes directory
os.chdir("/content")

# Generates TFRecord for the training image files
!python models/research/object_detection/dataset_tools/create_pascal_tf_record.py \
    --label_map_path=/content/models/research/object_detection/data/pascal_label_map.pbtxt \
    --data_dir=/content/VOCdevkit \
    --year=VOC2007 \
    --set=train \
    --output_path=/content/pascal_train.record 

# Generates TFRecord for the validation image files
!python models/research/object_detection/dataset_tools/create_pascal_tf_record.py \
    --label_map_path=/content/models/research/object_detection/data/pascal_label_map.pbtxt \
    --data_dir=/content/VOCdevkit \
    --year=VOC2007 \
    --set=val \
    --output_path=/content/pascal_val.record

## Downloading Pretrained Model
Based on the model selected at the top of this notebook, downloads the model and extracts its content.

In [ ]:
%cd /content/models/research

# The name of the object detection model to be used
MODEL = MODELS_CONFIG[selected_model]['model_name']

# The name of the selected pipeline file
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# The name of the tar file that contains the model
MODEL_FILE = MODEL + '.tar.gz'

# Base link that contains pretrained models
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# The name of the destination folder where the model will be saved
fine_tune_dir = '/content/models/research/pretrained_model'

# Downloads the selected model
if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

# Extracts the content of the tar file
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

# Changes the name of the folder that contains the downloaded model
os.remove(MODEL_FILE)
if (os.path.exists(fine_tune_dir)):
    shutil.rmtree(fine_tune_dir)
os.rename(MODEL, fine_tune_dir)

/content/models/research


In [ ]:
# Checks the content of the folder that contains the pretrained model
!echo {fine_tune_dir}
!ls -alh {fine_tune_dir}

## Configuring Training Pipeline
1. Sets the paths for the TFRecord files and label map file, and sets the batch_size, num_steps, num_classes, num_examples, num_visualizations and max_evals parameters in the configuration file.
2. Configures model_main.py file to save and evaluate the model at every 2500 steps.
3. Creates a folder to save the model at each checkpoint while training. 

In [ ]:
# The folder where the model will be saved at each checkpoint while training
model_dir = 'training/'

# Removes the content in the folder to fresh start
!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

In [ ]:

# The path to the folder containing all the sample config files
CONFIG_BASE = "/content/models/research/object_detection/samples/configs/"

# Path to the specified model's config file
model_pipeline = os.path.join(CONFIG_BASE, pipeline_file)
print(model_pipeline)

# Path to the model_main.py file
model_main_config = "/content/models/research/object_detection/model_main.py"
print(model_main_config)

/content/models/research/object_detection/samples/configs/rfcn_resnet101_coco.config
/content/models/research/object_detection/model_main.py


In [ ]:
%%writefile {model_pipeline}
# R-FCN with Resnet-101 (v1),  configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  faster_rcnn {
    num_classes: 20
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_resnet101'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    second_stage_box_predictor {
      rfcn_box_predictor {
        conv_hyperparams {
          op: CONV
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            truncated_normal_initializer {
              stddev: 0.01
            }
          }
        }
        crop_height: 18
        crop_width: 18
        num_spatial_bins_height: 3
        num_spatial_bins_width: 3
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.0
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 300
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
  }
}

train_config: {
  batch_size: 1
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0003
          schedule {
            step: 900000
            learning_rate: .00003
          }
          schedule {
            step: 1200000
            learning_rate: .000003
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  gradient_clipping_by_norm: 10.0
  fine_tune_checkpoint: "/content/models/research/pretrained_model/model.ckpt"
  from_detection_checkpoint: true
  # The number of steps the model will be trained on, we will change this parameter when using model_main.py
  num_steps: 100
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
}

train_input_reader: {
  tf_record_input_reader {
    input_path: "/content/pascal_train.record"
  }
  label_map_path: "/content/models/research/object_detection/data/pascal_label_map.pbtxt"
}

eval_config: {
  # The number of images in validation set
  num_examples: 2510
  # The number of images to display in TensorBoard while testing model's performance
  num_visualizations: 20
  # The number of evalutions executed at the training phase, we will remove this parameter to evaluate indefinitely
  #max_evals: 1
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "/content/pascal_val.record"
  }
  label_map_path: "/content/models/research/object_detection/data/pascal_label_map.pbtxt"
  shuffle: false
  num_readers: 1
}

Overwriting /content/models/research/object_detection/samples/configs/rfcn_resnet101_coco.config


In [ ]:
%%writefile {model_main_config}
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Binary to run train and evaluation on object detection model."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import flags

import tensorflow.compat.v1 as tf

from object_detection import model_lib

flags.DEFINE_string(
    'model_dir', None, 'Path to output model directory '
    'where event and checkpoint files will be written.')
flags.DEFINE_string('pipeline_config_path', None, 'Path to pipeline config '
                    'file.')
flags.DEFINE_integer('num_train_steps', None, 'Number of train steps.')
flags.DEFINE_boolean('eval_training_data', False,
                     'If training data should be evaluated for this job. Note '
                     'that one call only use this in eval-only mode, and '
                     '`checkpoint_dir` must be supplied.')
flags.DEFINE_integer('sample_1_of_n_eval_examples', 1, 'Will sample one of '
                     'every n eval input examples, where n is provided.')
flags.DEFINE_integer('sample_1_of_n_eval_on_train_examples', 5, 'Will sample '
                     'one of every n train input examples for evaluation, '
                     'where n is provided. This is only used if '
                     '`eval_training_data` is True.')
flags.DEFINE_string(
    'checkpoint_dir', None, 'Path to directory holding a checkpoint.  If '
    '`checkpoint_dir` is provided, this binary operates in eval-only mode, '
    'writing resulting metrics to `model_dir`.')
flags.DEFINE_boolean(
    'run_once', False, 'If running in eval-only mode, whether to run just '
    'one round of eval vs running continuously (default).'
)
flags.DEFINE_integer(
    'max_eval_retries', 0, 'If running continuous eval, the maximum number of '
    'retries upon encountering tf.errors.InvalidArgumentError. If negative, '
    'will always retry the evaluation.'
)
FLAGS = flags.FLAGS


def main(unused_argv):
  flags.mark_flag_as_required('model_dir')
  flags.mark_flag_as_required('pipeline_config_path')
  # Changes the RunConfig to save the model at every 2500 steps.
  config = tf.estimator.RunConfig(model_dir=FLAGS.model_dir, save_checkpoints_steps=2500)

  train_and_eval_dict = model_lib.create_estimator_and_inputs(
      run_config=config,
      pipeline_config_path=FLAGS.pipeline_config_path,
      train_steps=FLAGS.num_train_steps,
      sample_1_of_n_eval_examples=FLAGS.sample_1_of_n_eval_examples,
      sample_1_of_n_eval_on_train_examples=(
          FLAGS.sample_1_of_n_eval_on_train_examples))
  estimator = train_and_eval_dict['estimator']
  train_input_fn = train_and_eval_dict['train_input_fn']
  eval_input_fns = train_and_eval_dict['eval_input_fns']
  eval_on_train_input_fn = train_and_eval_dict['eval_on_train_input_fn']
  predict_input_fn = train_and_eval_dict['predict_input_fn']
  train_steps = train_and_eval_dict['train_steps']

  if FLAGS.checkpoint_dir:
    if FLAGS.eval_training_data:
      name = 'training_data'
      input_fn = eval_on_train_input_fn
    else:
      name = 'validation_data'
      # The first eval input will be evaluated.
      input_fn = eval_input_fns[0]
    if FLAGS.run_once:
      estimator.evaluate(input_fn,
                         steps=None,
                         checkpoint_path=tf.train.latest_checkpoint(
                             FLAGS.checkpoint_dir))
    else:
      model_lib.continuous_eval(estimator, FLAGS.checkpoint_dir, input_fn,
                                train_steps, name, FLAGS.max_eval_retries)
  else:
    train_spec, eval_specs = model_lib.create_train_and_eval_specs(
        train_input_fn,
        eval_input_fns,
        eval_on_train_input_fn,
        predict_input_fn,
        train_steps,
        eval_on_train_data=False)

    # Currently only a single Eval Spec is allowed.
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_specs[0])


if __name__ == '__main__':
  tf.app.run()

Overwriting /content/models/research/object_detection/model_main.py


## Using TensorBoard
1. Loads the TensorBoard notebook extension.
2. Starts TensorBoard for visualizing training and validation scores.

In [ ]:
# Loads the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
# Runs TensorBoard to visualize training scores
%tensorboard --logdir "/content/models/research/training"

## Training
Training the model. Read comments at the top of the cells carefully before running cells.

In [ ]:
# Mounts Drive to save the model checkpoint to Drive.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# I had some problems using model_main.py directly. For this reason, first we will
# create a checkpoint, then we will use model_main.py.

!python /content/models/research/object_detection/legacy/train.py \
    --logtostderr \
    --train_dir={model_dir} \
    --pipeline_config_path={model_pipeline} \

In [ ]:
# Loads the model checkpoint from Drive to continue from where
# we left in the last session.

!rm -rf "/content/models/research/training/"*
!cp -r "/content/drive/My Drive/Colab Notebooks/training_rfcn/"* "/content/models/research/training/"

In [ ]:
# Begins training.

!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={model_pipeline}\
    --num_train_steps=50000 \
    --model_dir={model_dir}\
    --sample_1_of_n_eval_examples=10 \
    --alsologtostderr \

In [ ]:
# Creates a directory to save the model checkpoint to Drive and deletes old files

!rm -rf "/content/drive/My Drive/Colab Notebooks/training_rfcn/"
os.makedirs("/content/drive/My Drive/Colab Notebooks/training_rfcn/", exist_ok=True)

# Saves the model checkpoint to Drive

!cp -r "/content/models/research/training/"* "/content/drive/My Drive/Colab Notebooks/training_rfcn/"

## Exporting the Trained Model
After all training sessions are completed, we will export the trained model.



In [ ]:
import numpy as np
import re

# The location where the exported model will be saved in
output_directory = '/content/models/research/fine_tuned_model'

# Goes through the folder where the model checkpoints are saved and picks the last model checkpoint.
lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')
last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)

# Exports the inference graph of the model
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={model_pipeline} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

In [ ]:
# Creates a directory to save the trained model to Drive and deletes old files

!rm -rf "/content/drive/My Drive/Colab Notebooks/model_rfcn/"
os.makedirs("/content/drive/My Drive/Colab Notebooks/model_rfcn/", exist_ok=True)

# Saves the trained model to Drive

trained_model = os.path.join(output_directory, "frozen_inference_graph.pb")
label_map_file = "/content/models/research/object_detection/data/pascal_label_map.pbtxt"
shutil.copy(trained_model, "/content/drive/My Drive/Colab Notebooks/model_rfcn/")
shutil.copy(label_map_file, "/content/drive/My Drive/Colab Notebooks/model_rfcn/")

'/content/drive/My Drive/Colab Notebooks/model_rfcn/pascal_label_map.pbtxt'